Install & Import Libraries

In [ ]:
!pip install -q tensorflow

import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.metrics import classification_report, confusion_matrix

Upload & Unzip Dataset

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mv "kaggle (1) (1).json" kaggle.json

Setup Kaggle in Colab


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Download Dataset

In [ ]:
!kaggle datasets download -d emmarex/plantdisease

unzip

In [ ]:
!unzip plantdisease.zip

**Explore Dataset Structure**

Check Folder Structure

In [ ]:
dataset_path = "/content/PlantVillage"
IMG_SIZE = 224
BATCH_SIZE = 32
print("Total Classes:", len(os.listdir(dataset_path)))
print("\nClass Names:\n")

for class_name in os.listdir(dataset_path):
    print(class_name)

Count Images Per Class

In [ ]:
class_counts = {}

for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    class_counts[class_name] = len(os.listdir(class_path))

print("Image count per class:\n")
for key, value in class_counts.items():
    print(f"{key} : {value}")

Plot Class Distribution

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.bar(class_counts.keys(), class_counts.values())
plt.xticks(rotation=90)
plt.title("Class Distribution")
plt.xlabel("Classes")
plt.ylabel("Number of Images")
plt.show()

Display Random Images From Different Classes

In [ ]:
import random
import cv2

plt.figure(figsize=(15,10))

for i, class_name in enumerate(os.listdir(dataset_path)[:6]):
    class_path = os.path.join(dataset_path, class_name)
    image_name = random.choice(os.listdir(class_path))
    image_path = os.path.join(class_path, image_name)

    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.subplot(2,3,i+1)
    plt.imshow(img)
    plt.title(class_name)
    plt.axis("off")

plt.show()

Data Preprocessing & Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

num_classes = train_data.num_classes
print("Total Classes:", num_classes)

Build Transfer Learning Model

In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False  # Freeze initially

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Add Callbacks

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "best_model.keras",
    monitor='val_accuracy',
    save_best_only=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=2,
    verbose=1
)

Initial Training (Frozen Base)

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=[early_stop, checkpoint, reduce_lr]
)

Fine-Tuning

In [ ]:
print("Starting Fine-Tuning...")

base_model.trainable = True

# Freeze first layers, train last 20 layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)

Plot Accuracy & Loss

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Initial Training Accuracy")
plt.legend(["Train", "Validation"])
plt.show()

plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title("Fine-Tuning Accuracy")
plt.legend(["Train", "Validation"])
plt.show()

Final Evaluation

In [ ]:
loss, acc = model.evaluate(val_data)
print("Final Validation Accuracy:", acc)

Confusion Matrix & Report

In [ ]:
val_data.reset()
pred = model.predict(val_data)
pred_classes = np.argmax(pred, axis=1)

true_classes = val_data.classes
class_labels = list(val_data.class_indices.keys())

print(classification_report(true_classes, pred_classes, target_names=class_labels))

In [ ]:
cm = confusion_matrix(true_classes, pred_classes)

plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

Save Final Model

In [ ]:
model.save("plant_disease_final_model.keras")

Predict on New Image

In [ ]:
from google.colab import files
files.upload()

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = "test.jpg"  # change name
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img)
img_array = preprocess_input(img_array)
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

predicted_index = np.argmax(prediction)
confidence = np.max(prediction) * 100

predicted_class = class_labels[predicted_index]

print("Predicted Disease:", predicted_class)
print("Confidence: {:.2f}%".format(confidence))

plt.imshow(img)
plt.axis("off")
plt.title(predicted_class)
plt.show()